# 2 - Artifact - Project Account Detail Batch Setting 

2. 1 - Overview

This notebook deploys a contract for account detail setting and than read key/values pairs for each account at `datatase/accounts.csv` and set account details in batch.

In [ ]:
from Crypto.Hash import keccak
import os
import binascii
from iroha import IrohaCrypto
from iroha import Iroha, IrohaGrpc
from iroha.ed25519 import H
import integration_helpers
from iroha.primitive_pb2 import can_set_my_account_detail
import sys
import csv
import json



if sys.version_info[0] < 3:
    raise Exception("Python 3 or a more recent version is required.")

# Load configuration from config.json file
config_path = "config.json"  # Update this path as needed
with open(config_path, "r") as f:
    config = json.load(f)

IROHA_HOST_ADDR = config["IROHA_HOST_ADDR"]
IROHA_PORT = config["IROHA_PORT"]
ADMIN_ACCOUNT_ID = config["ADMIN_ACCOUNT_ID"]
ADMIN_PRIVATE_KEY = config["ADMIN_PRIVATE_KEY"]

iroha = Iroha(ADMIN_ACCOUNT_ID)
net = IrohaGrpc("{}:{}".format(IROHA_HOST_ADDR, IROHA_PORT))

@integration_helpers.trace
def create_contract():
    bytecode = "608060405234801561001057600080fd5b5073a6abc17819738299b3b2c1ce46d55c74f04e290c6000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610b4c806100746000396000f3fe608060405234801561001057600080fd5b506004361061004c5760003560e01c80635bdb3a41146100515780637949a1b31461006f578063b7d66df71461009f578063d4e804ab146100cf575b600080fd5b6100596100ed565b6040516100669190610879565b60405180910390f35b61008960048036038101906100849190610627565b61024c565b6040516100969190610879565b60405180910390f35b6100b960048036038101906100b49190610693565b6103bb565b6040516100c69190610879565b60405180910390f35b6100d761059b565b6040516100e4919061085e565b60405180910390f35b606060006040516024016040516020818303038152906040527f5bdb3a41000000000000000000000000000000000000000000000000000000007bffffffffffffffffffffffffffffffffffffffffffffffffffffffff19166020820180517bffffffffffffffffffffffffffffffffffffffffffffffffffffffff8381831617835250505050905060008060008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16836040516101be9190610830565b600060405180830381855af49150503d80600081146101f9576040519150601f19603f3d011682016040523d82523d6000602084013e6101fe565b606091505b509150915081610243576040517f08c379a000000000000000000000000000000000000000000000000000000000815260040161023a9061091e565b60405180910390fd5b80935050505090565b60606000838360405160240161026392919061089b565b6040516020818303038152906040527f7949a1b3000000000000000000000000000000000000000000000000000000007bffffffffffffffffffffffffffffffffffffffffffffffffffffffff19166020820180517bffffffffffffffffffffffffffffffffffffffffffffffffffffffff8381831617835250505050905060008060008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168360405161032a9190610830565b600060405180830381855af49150503d8060008114610365576040519150601f19603f3d011682016040523d82523d6000602084013e61036a565b606091505b5091509150816103af576040517f08c379a00000000000000000000000000000000000000000000000000000000081526004016103a69061091e565b60405180910390fd5b80935050505092915050565b606060008484846040516024016103d4939291906108d2565b6040516020818303038152906040527fb7d66df7000000000000000000000000000000000000000000000000000000007bffffffffffffffffffffffffffffffffffffffffffffffffffffffff19166020820180517bffffffffffffffffffffffffffffffffffffffffffffffffffffffff8381831617835250505050905060008060008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168360405161049b9190610830565b600060405180830381855af49150503d80600081146104d6576040519150601f19603f3d011682016040523d82523d6000602084013e6104db565b606091505b509150915081610520576040517f08c379a00000000000000000000000000000000000000000000000000000000081526004016105179061091e565b60405180910390fd5b8460405161052e9190610847565b6040518091039020866040516105449190610847565b60405180910390208860405161055a9190610847565b60405180910390207f5e1b38cd47cf21b75d5051af29fa321eedd94877db5ac62067a076770eddc9d060405160405180910390a48093505050509392505050565b60008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1681565b60006105d26105cd84610963565b61093e565b9050828152602081018484840111156105ea57600080fd5b6105f5848285610a14565b509392505050565b600082601f83011261060e57600080fd5b813561061e8482602086016105bf565b91505092915050565b6000806040838503121561063a57600080fd5b600083013567ffffffffffffffff81111561065457600080fd5b610660858286016105fd565b925050602083013567ffffffffffffffff81111561067d57600080fd5b610689858286016105fd565b9150509250929050565b6000806000606084860312156106a857600080fd5b600084013567ffffffffffffffff8111156106c257600080fd5b6106ce868287016105fd565b935050602084013567ffffffffffffffff8111156106eb57600080fd5b6106f7868287016105fd565b925050604084013567ffffffffffffffff81111561071457600080fd5b610720868287016105fd565b9150509250925092565b610733816109e2565b82525050565b600061074482610994565b61074e81856109aa565b935061075e818560208601610a23565b61076781610ab6565b840191505092915050565b600061077d82610994565b61078781856109bb565b9350610797818560208601610a23565b80840191505092915050565b60006107ae8261099f565b6107b881856109c6565b93506107c8818560208601610a23565b6107d181610ab6565b840191505092915050565b60006107e78261099f565b6107f181856109d7565b9350610801818560208601610a23565b80840191505092915050565b600061081a6027836109c6565b915061082582610ac7565b604082019050919050565b600061083c8284610772565b915081905092915050565b600061085382846107dc565b915081905092915050565b6000602082019050610873600083018461072a565b92915050565b600060208201905081810360008301526108938184610739565b905092915050565b600060408201905081810360008301526108b581856107a3565b905081810360208301526108c981846107a3565b90509392505050565b600060608201905081810360008301526108ec81866107a3565b9050818103602083015261090081856107a3565b9050818103604083015261091481846107a3565b9050949350505050565b600060208201905081810360008301526109378161080d565b9050919050565b6000610948610959565b90506109548282610a56565b919050565b6000604051905090565b600067ffffffffffffffff82111561097e5761097d610a87565b5b61098782610ab6565b9050602081019050919050565b600081519050919050565b600081519050919050565b600082825260208201905092915050565b600081905092915050565b600082825260208201905092915050565b600081905092915050565b60006109ed826109f4565b9050919050565b600073ffffffffffffffffffffffffffffffffffffffff82169050919050565b82818337600083830152505050565b60005b83811015610a41578082015181840152602081019050610a26565b83811115610a50576000848401525b50505050565b610a5f82610ab6565b810181811067ffffffffffffffff82111715610a7e57610a7d610a87565b5b80604052505050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052604160045260246000fd5b6000601f19601f8301169050919050565b7f4572726f722063616c6c696e67207365727669636520636f6e7472616374206660008201527f756e6374696f6e0000000000000000000000000000000000000000000000000060208201525056fea26469706673582212206ad40afbd4cc9c87ae154542d003c9538e4b89473a13cadd3cbf618ea181206864736f6c63430008040033"
    """Bytecode was generated using remix editor  https://remix.ethereum.org/ from file detail.sol. """
    tx = iroha.transaction(
        [iroha.command("CallEngine", caller=ADMIN_ACCOUNT_ID, input=bytecode)]
    )
    IrohaCrypto.sign_transaction(tx, ADMIN_PRIVATE_KEY)
    net.send_tx(tx)
    hex_hash = binascii.hexlify(IrohaCrypto.hash(tx))
    for status in net.tx_status_stream(tx):
        print(status)
    return hex_hash

@integration_helpers.trace
def set_account_detail(address, account, key, value):
    params = integration_helpers.get_first_four_bytes_of_keccak(
        b"setAccountDetail(string,string,string)"
    )
    no_of_param = 3
    for x in range(no_of_param):
        params = params + integration_helpers.left_padded_address_of_param(
            x, no_of_param
        )
    params = params + integration_helpers.argument_encoding(account)  # source account id
    params = params + integration_helpers.argument_encoding(key)  # key
    params = params + integration_helpers.argument_encoding(value)  #  value
    tx = iroha.transaction(
        [
            iroha.command(
                "CallEngine", caller=ADMIN_ACCOUNT_ID, callee=address, input=params
            )
        ]
    )
    IrohaCrypto.sign_transaction(tx, ADMIN_PRIVATE_KEY)
    response = net.send_tx(tx)
    print(response)
    for status in net.tx_status_stream(tx):
        print(status)
    hex_hash = binascii.hexlify(IrohaCrypto.hash(tx))
    return hex_hash

# - Deploys the contract
hash = create_contract()
address = integration_helpers.get_engine_receipts_address(hash)
integration_helpers.get_engine_receipts_result(hash)

In [22]:
import csv
import json

# Load the JSON data from the file
with open('datasets/projects_metadata.json') as f:
    projects = json.load(f)

# Open the CSV file and read it line by line
with open('datasets/projects.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    
    # Get the length of both files to ensure we don't go out of bounds
    json_length = len(projects)
    csv_reader = list(reader)  # Convert reader to a list so we can access its length

    # Iterate over each row in the CSV file
    for i, row in enumerate(csv_reader):
        project_id = row['project_id']
        print (project_id)
        
        # Get the corresponding entry from the JSON data
        if i < json_length:
            project_json = projects[i]
        else:
            break  # If we've reached the end of the JSON file, stop iterating over CSV rows
        
        # Display the JSON entry if found
        print(project_json)


{'title': 'Assessing the Impact of Urbanization on Avian Migration Patterns', 'abstract': 'This study aims to investigate the effects of urbanization on avian migration patterns and identify potential conservation strategies.', 'keywords': ['urbanization', 'avian migration', 'conservation'], 'start_date': '2020-01-01', 'end_date': '2022-12-31', 'funding_agency': 'National Science Foundation', 'location': 'San Francisco Bay Area, California, USA'}
{'title': 'Investigating the Effects of Climate Change on Coral Reef Ecosystems', 'abstract': 'This study aims to investigate the impacts of climate change on coral reef ecosystems and identify potential mitigation strategies.', 'keywords': ['climate change', 'coral reefs', 'ecosystem resilience'], 'start_date': '2019-06-01', 'end_date': '2021-05-31', 'funding_agency': 'National Oceanic and Atmospheric Administration', 'location': 'Great Barrier Reef, Australia'}
{'title': 'Understanding the Role of Microplastics in Marine Food Webs', 'abstrac

In [ ]:
# - Reads the accounts dataset and set account_id and additional key/value pairs
with open('datasets/projects.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        project_id = row['project_id']
        print(f"{project_id}")

        
# hash = set_account_detail(address, account_id, key, value)

In [ ]:
#For metadata handling
from json_processing import display_json_data

display_json_data('datasets/projects_metadata.json')

In [ ]:
import csv
import json

file_path = "datasets/projects.csv"

# Open the CSV file in read mode
with open(file_path, 'r') as csvfile:
    reader = csv.reader(csvfile)

    # Iterate over each line (project_id) in the CSV file
    for project_id in reader:
        # Assuming the JSON file is named after the project_id with a .json extension
        json_file_path = "datasets/projects_metadata.json"

        try:
            # Attempt to load the JSON file as a Python dictionary
            with open(json_file_path, 'r') as jsonfile:
                json_dict = json.load(jsonfile)

                # Print the contents of the JSON file as a dictionary
                print(json.dumps(json_dict, indent=4))
        except FileNotFoundError:
            print(f"Error: JSON file '{json_file_path}' not found.")
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON file '{json_file_path}': {e}")




In [18]:
if isinstance(json_dict, list):
                print("List of dictionaries:")
                for i, item in enumerate(json_dict, start=1):  
                    if isinstance(item, dict):  
                        print(f"Dictionary {i}:")
                        # print(json.dumps(item, indent=4))  # Display each dictionary as formatted JSON
                        project_metadata = json.dumps(item)
                        print(project_metadata)
                        # project_metadata_cid = upload_json_to_ipfs(project_metadata)
                        # print(project_metadata_cid)
                        
                    elif isinstance(json_dict, dict):
                        print("Dictionary:")
                        print(json.dumps(data, indent=4))  # Display the single dictionary as formatted JSON
            

List of dictionaries:
Dictionary 1:
{"title": "Assessing the Impact of Urbanization on Avian Migration Patterns", "abstract": "This study aims to investigate the effects of urbanization on avian migration patterns and identify potential conservation strategies.", "keywords": ["urbanization", "avian migration", "conservation"], "start_date": "2020-01-01", "end_date": "2022-12-31", "funding_agency": "National Science Foundation", "location": "San Francisco Bay Area, California, USA"}
Dictionary 2:
{"title": "Investigating the Effects of Climate Change on Coral Reef Ecosystems", "abstract": "This study aims to investigate the impacts of climate change on coral reef ecosystems and identify potential mitigation strategies.", "keywords": ["climate change", "coral reefs", "ecosystem resilience"], "start_date": "2019-06-01", "end_date": "2021-05-31", "funding_agency": "National Oceanic and Atmospheric Administration", "location": "Great Barrier Reef, Australia"}
Dictionary 3:
{"title": "Unders

In [ ]:
        metadata_index = 0
        file_index = 0

        while metadata_index < len(metadata_cids) and file_index < len(file_cids):
            variable_1 = f"file_{metadata_index+1}_metadata_CID"
            variable_2 = metadata_cids[metadata_index]
            print(f"{variable_1} = {variable_2}")
            metadata_index += 1
            hash = set_account_detail(address, account, variable_1, variable_2)
            print(account)

            if metadata_index <= len(metadata_cids) and file_index < len(file_cids):
                variable_1 = f"file_{metadata_index}_CID"
                variable_2 = file_cids[file_index]
                print(f"{variable_1} = {variable_2}")
                file_index += 1
                hash = set_account_detail(address, account, variable_1, variable_2)
                print(account)